In [1]:
!pip install -r requirements.txt -q


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from elasticsearch import Elasticsearch, helpers
from helpers.runtime import AlquimiaRuntime
import json

In [3]:
ALQUIMIA_RUNTIME_URL = os.environ.get("ALQUIMIA_RUNTIME_URL")
ALQUIMIA_TOKEN = os.environ.get("ALQUIMIA_TOKEN")
S3_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID')
S3_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
S3_BUCKET_NAME = os.environ.get("AWS_S3_BUCKET")
S3_ENDPOINT = os.environ.get("AWS_S3_ENDPOINT")
ELASTIC_URL = os.environ.get('ELASTIC_URL')
ELASTIC_AUTH = [os.environ.get('ELASTIC_AUTH_USER'), os.environ.get('ELASTIC_AUTH_PASSWORD')]
dataset = os.environ.get("dataset", "asb")
runtime_index = f"{dataset}-runtime"

In [4]:
runtime = AlquimiaRuntime(ALQUIMIA_RUNTIME_URL, ALQUIMIA_TOKEN)
conversations = runtime.infer_dataset(dataset)

2025-04-09 13:11:40,405 - INFO - [RUNTIME] Starting to infer dataset
2025-04-09 13:11:40,406 - INFO - [RUNTIME] It must use force_profile: {'_collection_id': 'asb-empresas'}
2025-04-09 13:11:40,406 - INFO - [RUNTIME/aeb1cdce-8e69-4975-879f-daa13cedda7a] Starting Conversation
2025-04-09 13:11:45,451 - INFO - [RUNTIME/aeb1cdce-8e69-4975-879f-daa13cedda7a] Human: Give me summary statistics of loan amounts.

Thinkings:
{'tool_summary': {'name': 'query_dataframe', 'parameters': {'sql_query': 'SELECT COUNT(*) AS total_loans, SUM(LOAN_AMOUNT) AS total_amount, AVG(LOAN_AMOUNT) AS average_amount, MIN(LOAN_AMOUNT) AS minimum_amount, MAX(LOAN_AMOUNT) AS maximum_amount FROM loans;'}}, 'tool_output': {'tool_call_id': '2be7b40930ad4c2593447d8b3f29fadb', 'status': 'success', 'message': None, 'result': 'total_loans,total_amount,average_amount,minimum_amount,maximum_amount\n20,8450000,422500.0,200000,700000\n'}}

Assistant: The total number of loans is 20, with a total amount of $8,450,000. The average

In [ ]:
mapping_runtime = {
  "mappings": {
    "properties": {
      "session_id": {"type": "keyword"},
      "observation": {"type": "text"},
      "assistant": {"type": "text"},
      "ground_truth_assistant": {"type": "text"},
      "question": {"type": "text"},
      "qa_id": {"type": "keyword"},
      "assistant_id": {"type":"keyword"}
    }
  }
}

In [ ]:
es = Elasticsearch(
    ELASTIC_URL,
    basic_auth=tuple(ELASTIC_AUTH),
)

In [ ]:
docs = []
for conversation in conversations:
    for batch in conversation['conversation']:
        docs.append({
            "_index": runtime_index,
            "_source": {
                "session_id": conversation['session_id'],
                "observation": batch['observation'] if batch['observation'] else "",
                "assistant": batch['assistant'],
                "ground_truth_assistant": batch['ground_truth_assistant'] if batch['ground_truth_assistant'] else "",
                "question": batch['question'],
                "qa_id": batch['qa_id'],
                "assistant_id": conversation['assistant_id']
            }
        })

In [ ]:
def recreate_index(index_name: str, mapping: dict):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created.")

recreate_index(runtime_index, mapping_runtime)

In [ ]:
helpers.bulk(es, docs)

In [5]:
with open("dataset.json", "w") as outfile:
    json.dump(conversations, outfile, indent=4)